# Code for running inference

To convert this to a script, should have a name for the whole training task which goes in dataset.json and/or datalist.json because this will help for the task.json needed for inference.

Look into the minimal amount of metaata monai requires in the form of a task.json or other struct to initialize the builder and then run the ensemble

In [1]:
import os
from pathlib import Path
import numpy as np
import json

import mri_data
import monai_training

from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys

# from reload_recursive import reload_recursive

In [2]:
# reload_recursive(mri_data)
# reload_recursive(monai_training)
from mri_data import utils
from mri_data.file_manager import scan_3Tpioneer_bids, Scan, DataSet
from monai_training import preprocess

**Edit These**

In [ ]:
#! Set these variables
work_dir_name = "cp_work_dir_pituitary1"
train_dataset_file_name = "training-dataset-desktop1.json"
prediction_postfix = "pituitary_pred"
modalities = ["t1"]
save_dir = Path("/media/smbshare/3Tpioneer_bids_predictions")

Set paths

In [ ]:
projects_root = Path("/home/hemondlab/Dev")
drive_root = Path("/media/smbshare")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
train_dataset_file = work_dir / train_dataset_file_name

prediction_filename = (
    ".".join(sorted(modalities)) + "_" + prediction_postfix + ".nii.gz"
)

Get all the scans that were not in the training and testing set to create the inference dataset

In [18]:
# the scans that were used in the training 
dataset_train, _ = preprocess.load_dataset(train_dataset_file)

# dataset_train2 has the same subject/sessions that are in dataset_train but with a subset of the keys
#   so that they can be compared to scans in the full data set when getting the set difference
dataset_proc = preprocess.DataSetProcesser.new_dataset(dataroot, scan_3Tpioneer_bids, filters=["first_ses"])
dataset_full = dataset_proc.dataset
dataset_train2 = DataSet.dataset_like(dataset_train, ['subid', 'sesid'])
dataset_inference = DataSet.from_scans(set(dataset_full) - set(dataset_train2))

Prepare the images in the inference dataset (i.e combine flair and t1 images)

In [ ]:
dataset_proc = preprocess.DataSetProcesser(dataset_inference)
dataset_proc.prepare_images(["flair", "t1"])

In [23]:
images = []
for scan in dataset_proc.dataset:
    infile: Path = scan.image_path
    images.append({"image": str(infile.relative_to(dataset_proc.dataset.dataroot))})

datalist = {
    "testing": images
}

datalist_file = work_dir / "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f)

task = {
  "name": "infer_pineal",
  "task": "segmentation",
  "modality": "MRI",
  "datalist": str(work_dir / "datalist.json"),
  "dataroot": str(dataroot),
}

task_file = os.path.join(work_dir, "inference-task.json")
with open(task_file, "w") as f:
    json.dump(task, f)

In [8]:
input_cfg = work_dir  / "inference-task.json"  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)

In [ ]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage", 
    "output_dir": save_dir, 
    "data_root_dir": dataroot, 
    "output_postfix": "pineal_pred", 
    "separate_folder": False}
                               
pred = ensemble(pred_param = {"image_save_func": save_params})